In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molsysmt as msm

# How to extract, fit, center and wrap to MIC a trajectory

In [3]:
trajectory = msm.demo['pentarginine']['traj.h5']

In [4]:
msm.info(trajectory)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_ions,n_peptides,n_structures
file:h5,4822,1561,1553,3,1553,None,1545,7,1,100


In [5]:
msm.view(trajectory, standard=True, with_water_as='licorice')

NGLWidget(max_frame=99)

## Extract

In [ ]:
molecular_system = '/home/diego/projects@dprada/CPP_permeability/peptide_preparation/traj.h5'

In [ ]:
msm.convert(molecular_system, to_form='last_100_frames.h5', structure_indices=list(range(79899, 79999)))

In [ ]:
#msm.convert(molecular_system, to_form='traj_ions_peptide.h5', selection='molecule_type!="water"', progress_bar=True)

## Fit, center, and wrap to MIC

### Using the molsysmt.basic.convert method

In [ ]:
molecular_system = msm.convert('last_100_frames.h5', fit='atom_name in ["CA", "C", "N"]',
                   center='atom_name in ["CA", "C", "N"]', wrap_to_mic=True)

In [ ]:
msm.view(molecular_system, standard=True, water_as_surface=True)

In [ ]:
msm.convert('last_100_frames.h5', to_form='fit_center_wrapped.h5', selection='molecule_type!="water"',
            fit='atom_name in ["CA", "C", "N"]', center='atom_name in ["CA", "C", "N"]',
            wrap_to_mic=True)

### Working with the native molsysmt.MolSys form

In [ ]:
molecular_system = msm.convert('last_100_frames.h5')

In [ ]:
molecular_system = msm.structure.fit(molecular_system, selection='atom_name in ["CA", "C", "N"]')
molecular_system = msm.structure.center(molecular_system, center_of_selection='atom_name in ["CA", "C", "N"]')
molecular_system = msm.pbc.wrap_to_mic(molecular_system)

In [ ]:
msm.info(molecular_system)

In [ ]:
msm.view(molecular_system, standard=True, water_as_surface=True)

### Working with an iterator

In [ ]:
iterator = msm.Iterator('last_100_frames.h5')

In [ ]:
final_molecular_system = None

In [ ]:
for molecular_system in iterator:
    molecular_system = msm.structure.fit(aux_molecular_system, selection='atom_name in ["CA", "C", "N"]')
    molecular_system = msm.structure.center(molecular_system, center_of_selection='atom_name in ["CA", "C", "N"]')
    molecular_system = msm.pbc.wrap_to_mic(molecular_system)
    final_molecular_system = msm.append_structures(final_molecular_system, molecular_system)

In [ ]:
msm.info(final_molecular_system)

In [ ]:
msm.view(final_molecular_system, standard=True, water_as_surface=True)

There is no need to work with an iterator and a molsysmt.MolSys object. You can explore other possilibities like iterating only the coordinates and boxes:

In [ ]:
topology = msm.convert('last_100_frames.h5', 'molsysmt.Topology')

all_coordinates = []
all_boxes = []

iterator = msm.Iterator('last_100_frames.h5', coordinates=True, box=True)

for coordinates, box in iterator:
    molecular_system = [topology, coordinates, box]
    molecular_system = msm.structure.fit(molecular_system, selection='atom_name in ["CA", "C", "N"]')
    molecular_system = msm.structure.center(molecular_system, center_of_selection='atom_name in ["CA", "C", "N"]')
    molecular_system = msm.pbc.wrap_to_mic(molecular_system)
    new_coordinates = msm.get(molecular_system, coordinates=True) # new_coordinates = molecular_system[1]
    all_coordinates.append(new_coordinates)
    all_box.append(box)

In [ ]:
msm.info(molecular_system)

In [ ]:
msm.view(molecular_system, standard=True, water_as_surface=True)

Or:

In [ ]:
atoms_to_fit_and_center = msm.select('last_100_frames.h5', selection='atom_name in ["CA", "C", "N"]')

all_coordinates = []
all_boxes = []

iterator = msm.Iterator('last_100_frames.h5', coordinates=True, box=True)

for coordinates, box in iterator:
    coordinates = msm.structure.fit(coordinates, selection=atoms_to_fit_and_center)
    coordinates = msm.structure.center(coordinates, center_of_selection=atoms_to_fit_and_center)
    coordinates, _ = msm.pbc.wrap_to_mic([coordinates, box])
    all_coordinates.append(coordinates)
    all_boxes.append(box)

In [ ]:
msm.view(['last_100_frames.h5', coordinates, boxes], standard=True, water_as_surface=True)